In [1]:
# -*- coding: utf-8 -*-

"""
EVCのためのEV-GMMを構築します. そして, 適応学習する.
詳細 : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580aabf534c4dbb8bc.pdf

This program make EV-GMM for EVC. Then, it make adaptation learning.
Check detail : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580abf534c4dbb8bc.pdf
"""

'\nEVCのためのEV-GMMを構築します. そして, 適応学習する.\n詳細 : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580aabf534c4dbb8bc.pdf\n\nThis program make EV-GMM for EVC. Then, it make adaptation learning.\nCheck detail : https://pdfs.semanticscholar.org/cbfe/71798ded05fb8bf8674580abf534c4dbb8bc.pdf\n'

In [2]:
from __future__ import division, print_function

import os
from shutil import rmtree
import argparse
import glob
import pickle
import time

import numpy as np
from numpy.linalg import norm 
from sklearn.decomposition import PCA
from sklearn.mixture import GMM # sklearn 0.20.0から使えない
from sklearn.preprocessing import StandardScaler
import scipy.signal
import scipy.sparse

%matplotlib inline
import matplotlib.pyplot as plt
import IPython 
from IPython.display import Audio 

import soundfile as sf
import wave 
import pyworld as pw
import librosa.display
import pysptk

from dtw import dtw
import warnings
warnings.filterwarnings('ignore')

In [3]:
"""
Parameters

__Mixtured : GMM混合数
__versions : 実験セット
__convert_source : 変換元話者のパス
__convert_target : 変換先話者のパス
"""
# parameters 
__Mixtured = 40
__versions = 'pre-stored0.1.2'
__convert_source = 'input/EJM10/V01/T01/TIMIT/000/*.wav' 
__convert_target = 'adaptation/EJF01/V01/T01/ATR503/A/*.wav'

# settings
__same_path = './utterance/' + __versions + '/'
__output_path = __same_path + 'output/EJF01/' # EJF01, EJF07, EJM04, EJM05

Mixtured = __Mixtured
pre_stored_pickle = __same_path + __versions + '.pickle'
pre_stored_source_list = __same_path + 'pre-source/**/V01/T01/**/*.wav'
pre_stored_list = __same_path + "pre/**/V01/T01/**/*.wav"
#pre_stored_target_list = "" (not yet)
pre_stored_gmm_init_pickle = __same_path + __versions + '_init-gmm.pickle'
pre_stored_sv_npy = __same_path + __versions + '_sv.npy'

save_for_evgmm_covarXX = __output_path + __versions + '_covarXX.npy'
save_for_evgmm_covarYX = __output_path + __versions + '_covarYX.npy'
save_for_evgmm_fitted_source = __output_path + __versions + '_fitted_source.npy'
save_for_evgmm_fitted_target = __output_path + __versions + '_fitted_target.npy'
save_for_evgmm_weights = __output_path + __versions + '_weights.npy'
save_for_evgmm_source_means = __output_path + __versions + '_source_means.npy'

for_convert_source = __same_path + __convert_source
for_convert_target = __same_path + __convert_target
converted_voice_npy = __output_path + 'sp_converted_' + __versions  
converted_voice_wav = __output_path + 'sp_converted_' + __versions 
mfcc_save_fig_png = __output_path + 'mfcc3dim_' + __versions 
f0_save_fig_png = __output_path + 'f0_converted' + __versions
converted_voice_with_f0_wav = __output_path + 'sp_f0_converted' + __versions

In [4]:
EPSILON = 1e-8

class MFCC:
    """
    MFCC() : メル周波数ケプストラム係数(MFCC)を求めたり、MFCCからスペクトルに変換したりするクラス.
    動的特徴量(delta)が実装途中.
    ref : http://aidiary.hatenablog.com/entry/20120225/1330179868
    """
    
    
    def __init__(self, frequency, nfft=1026, dimension=24, channels=24):
        """
        各種パラメータのセット
        nfft : FFTのサンプル点数
        frequency : サンプリング周波数
        dimension : MFCC次元数
        channles : メルフィルタバンクのチャンネル数(dimensionに依存)
        fscale : 周波数スケール軸
        filterbankl, fcenters : フィルタバンク行列, フィルタバンクの頂点(?)
        """
        self.nfft = nfft
        self.frequency = frequency
        self.dimension = dimension
        self.channels = channels
        self.fscale = np.fft.fftfreq(self.nfft, d = 1.0 / self.frequency)[: int(self.nfft / 2)]
        self.filterbank, self.fcenters = self.melFilterBank()
    
    def hz2mel(self, f):
        """
        周波数からメル周波数に変換
        """
        return 1127.01048 * np.log(f / 700.0 + 1.0)
    
    def mel2hz(self, m):
        """
        メル周波数から周波数に変換
        """     
        return 700.0 * (np.exp(m / 1127.01048) - 1.0)

    def melFilterBank(self):
        """
        メルフィルタバンクを生成する
        """      
        fmax = self.frequency / 2
        melmax = self.hz2mel(fmax)
        nmax = int(self.nfft / 2)
        df = self.frequency / self.nfft
        dmel = melmax / (self.channels + 1)
        melcenters = np.arange(1, self.channels + 1) * dmel
        fcenters = self.mel2hz(melcenters)
        indexcenter = np.round(fcenters / df)
        indexstart = np.hstack(([0], indexcenter[0:self.channels - 1]))
        indexstop = np.hstack((indexcenter[1:self.channels], [nmax]))

        filterbank = np.zeros((self.channels, nmax))
        for c in np.arange(0, self.channels):
            increment = 1.0 / (indexcenter[c] - indexstart[c])
            # np,int_ は np.arangeが[0. 1. 2. ..]となるのをintにする
            for i in np.int_(np.arange(indexstart[c], indexcenter[c])):
                filterbank[c, i] = (i - indexstart[c]) * increment
            decrement = 1.0 / (indexstop[c] - indexcenter[c])
            # np,int_ は np.arangeが[0. 1. 2. ..]となるのをintにする
            for i in np.int_(np.arange(indexcenter[c], indexstop[c])):
                filterbank[c, i] = 1.0 - ((i - indexcenter[c]) * decrement)

        return filterbank, fcenters
    
    def mfcc(self, spectrum):
        """
        スペクトルからMFCCを求める.
        """
        mspec = []
        mspec = np.log10(np.dot(spectrum, self.filterbank.T))
        mspec = np.array(mspec)
        
        return scipy.fftpack.realtransforms.dct(mspec, type=2, norm="ortho", axis=-1)
    
    def delta(self, mfcc):
        """
        MFCCから動的特徴量を求める.
        現在は,求める特徴量フレームtをt-1とt+1の平均としている.
        """
        mfcc = np.concatenate([
            [mfcc[0]], 
            mfcc, 
            [mfcc[-1]]
        ]) # 最初のフレームを最初に、最後のフレームを最後に付け足す
        delta = None
        for i in range(1, mfcc.shape[0] - 1):
            slope = (mfcc[i+1] - mfcc[i-1]) / 2
            if delta is None:
                delta = slope
            else:
                delta = np.vstack([delta, slope])
        return delta
    
    def imfcc(self, mfcc, spectrogram):
        """
        MFCCからスペクトルを求める.
        """
        im_sp = np.array([])
        for i in range(mfcc.shape[0]):
            mfcc_s = np.hstack([mfcc[i], [0] * (self.channels - self.dimension)])
            mspectrum = scipy.fftpack.idct(mfcc_s, norm='ortho')
            # splrep はスプライン補間のための補間関数を求める
            tck = scipy.interpolate.splrep(self.fcenters, np.power(10, mspectrum))
            # splev は指定座標での補間値を求める
            im_spectrogram = scipy.interpolate.splev(self.fscale, tck)
            im_sp = np.concatenate((im_sp, im_spectrogram), axis=0)
            
        return im_sp.reshape(spectrogram.shape)
            
    def trim_zeros_frames(x, eps=1e-7):
        """
        無音区間を取り除く.
        """
        T, D = x.shape
        s = np.sum(np.abs(x), axis=1)
        s[s < 1e-7] = 0.
        return x[s > eps]

In [5]:
def analyse_by_world_with_harverst(x, fs, fftl=1024, shiftms=5.0, minf0=40.0, maxf0=500.0):
    """
    WORLD音声分析合成器で基本周波数F0,スペクトル包絡,非周期成分を求める.
    基本周波数F0についてはharvest法により,より精度良く求める.
    """
    #f0, time_axis = pw.harvest(x, fs, frame_period=shiftms)
    f0, time_axis = pw.harvest(x, fs, f0_floor=minf0, f0_ceil=maxf0, frame_period=shiftms)
    spc = pw.cheaptrick(x, f0, time_axis, fs, fft_size=fftl)
    ap = pw.d4c(x, f0, time_axis, fs, fft_size=fftl)
    
    # 4 Harvest with F0 refinement (using Stonemask)
    """
    frame_period = 5
    _f0, t = pw.harvest(x, fs, frame_period=frame_period)
    f0 = pw.stonemask(x, _f0, t, fs)
    sp = pw.cheaptrick(x, f0, t, fs)
    ap = pw.d4c(x, f0, t, fs)
    """
    
    return f0, spc, ap

def wavread(file):
    """
    wavファイルから音声トラックとサンプリング周波数を抽出する.
    """
    wf = wave.open(file, "r")
    fs = wf.getframerate()
    x = wf.readframes(wf.getnframes())
    x = np.frombuffer(x, dtype= "int16") / 32768.0
    wf.close()
    return x, float(fs)

def preEmphasis(signal, p=0.97):
    """
    MFCC抽出のための高域強調フィルタ.
    波形を通すことで,高域成分が強調される.
    """
    return scipy.signal.lfilter([1.0, -p], 1, signal)

def alignment(source, target, path):
    """
    タイムアライメントを取る.
    target音声をsource音声の長さに合うように調整する.
    """
    # ここでは814に合わせよう(targetに合わせる)
    # p_p = 0 if source.shape[0] > target.shape[0] else 1

    #shapes = source.shape if source.shape[0] > target.shape[0] else target.shape 
    shapes = source.shape
    align = np.array([])
    for (i, p) in enumerate(path[0]):
        if i != 0:
            if j != p:
                temp = np.array(target[path[1][i]])
                align = np.concatenate((align, temp), axis=0)
        else:
            temp = np.array(target[path[1][i]])
            align = np.concatenate((align, temp), axis=0)   
        
        j = p
        
    return align.reshape(shapes)

In [6]:
"""
pre-stored学習のためのパラレル学習データを作る。
時間がかかるため、利用できるlearn-data.pickleがある場合はそれを利用する。
それがない場合は一から作り直す。
"""
timer_start = time.time()
dim = 24
alpha = 0.42
if os.path.exists(pre_stored_pickle):
    print("exist, ", pre_stored_pickle)
    with open(pre_stored_pickle, mode='rb') as f:
        total_data = pickle.load(f)
        print("open, ", pre_stored_pickle)
        print("Load pre-stored time = ", time.time() - timer_start , "[sec]")
else:

    source_mfcc = []
    #source_data_sets = []
    for name in sorted(glob.iglob(pre_stored_source_list, recursive=True)):
        print(name)
        x, fs = sf.read(name)
        f0, sp, ap = analyse_by_world_with_harverst(x, fs)
    
        mc = pysptk.sp2mc(sp, dim, alpha)
        #source_data = np.hstack([source_mfcc_temp, mfcc.delta(source_mfcc_temp)]) # static & dynamic featuers
        source_mfcc.append(mc)
        #source_data_sets.append(source_data)

    total_data = []
    
    i = 0
    _s_len = len(source_mfcc)
    for name in sorted(glob.iglob(pre_stored_list, recursive=True)):
        print(name, len(total_data))
        x, fs = sf.read(name)
        f0, sp, ap = analyse_by_world_with_harverst(x, fs)
        
        mc = pysptk.sp2mc(sp, dim, alpha)

        dist, cost, acc, path = dtw(source_mfcc[i%_s_len], mc, dist=lambda x, y: norm(x - y, ord=1))
        #print('Normalized distance between the two sounds:' + str(dist))
        #print("target_mfcc = {0}".format(target_mfcc.shape))

        aligned = alignment(source_mfcc[i%_s_len], mc, path)
        #target_data_sets = np.hstack([aligned, mfcc.delta(aligned)]) # static & dynamic features
        #learn_data = np.hstack((source_data_sets[i], target_data_sets))
        learn_data = np.hstack([source_mfcc[i%_s_len], aligned])

        total_data.append(learn_data)
        i += 1
        
    with open(pre_stored_pickle, 'wb') as output:
        pickle.dump(total_data, output)
        print("Make, ", pre_stored_pickle)
        print("Make pre-stored time = ", time.time() - timer_start , "[sec]")

./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A03.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A04.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A07.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A08.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A09.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A11.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A13.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A14.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A17.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A18.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A19.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A20.wav
./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A21.wav
./utterance/

./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A17.wav 72
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A18.wav 73
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A25.wav 74
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A29.wav 75
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A30.wav 76
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESUS/000/A39.wav 77
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A01.wav 78
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A02.wav 79
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A03.wav 80
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A04.wav 81
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A05.wav 82
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A06.wav 83
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A07.wav 84
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A08.wav 85
./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/TIMIT/000/A09.wav 86
.

./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A14.wav 191
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A17.wav 192
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A18.wav 193
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A19.wav 194
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A20.wav 195
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A21.wav 196
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESPBOBAMA1/000/A22.wav 197
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A02.wav 198
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A03.wav 199
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A05.wav 200
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A06.wav 201
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A07.wav 202
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A08.wav 203
./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/ESUS/000/A09.wav 204
./utterance/pre-stor

./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A03.wav 310
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A04.wav 311
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A05.wav 312
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A06.wav 313
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A07.wav 314
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A08.wav 315
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A09.wav 316
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A10.wav 317
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A12.wav 318
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A13.wav 319
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A15.wav 320
./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A16.wav 321
./utterance/pre-stored0.1.2/pre/EJF10/V01/T01/ESPBOBAMA1/000/A03.wav 322
./utterance/pre-stored0.1.2/pre/EJF10/V01/T01/ESPBOBAMA1/000/A04.wav 323
./utterance/pre-stored0.1.2/pre/EJF10/

./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A02.wav 428
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A03.wav 429
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A05.wav 430
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A06.wav 431
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A07.wav 432
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A08.wav 433
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A09.wav 434
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A10.wav 435
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A11.wav 436
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A13.wav 437
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A14.wav 438
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A16.wav 439
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A17.wav 440
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/ESUS/000/A18.wav 441
./utterance/pre-stored0.1.2/pre/EJF102/V01/T01/E

./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A10.wav 547
./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A12.wav 548
./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A13.wav 549
./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A15.wav 550
./utterance/pre-stored0.1.2/pre/EJM01/V01/T01/TIMIT/000/A16.wav 551
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A03.wav 552
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A04.wav 553
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A07.wav 554
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A08.wav 555
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A09.wav 556
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A11.wav 557
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A13.wav 558
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A14.wav 559
./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESPBOBAMA1/000/A17.wav 560
./u

./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A11.wav 666
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A13.wav 667
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A14.wav 668
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A16.wav 669
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A17.wav 670
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A18.wav 671
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A25.wav 672
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A29.wav 673
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A30.wav 674
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A39.wav 675
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A01.wav 676
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A02.wav 677
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A03.wav 678
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A04.wav 679
./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/TIMIT/000/A0

./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A08.wav 785
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A09.wav 786
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A11.wav 787
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A13.wav 788
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A14.wav 789
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A17.wav 790
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A18.wav 791
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A19.wav 792
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A20.wav 793
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A21.wav 794
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESPBOBAMA1/000/A22.wav 795
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESUS/000/A02.wav 796
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESUS/000/A03.wav 797
./utterance/pre-stored0.1.2/pre/EJM09/V01/T01/ESUS/000/A05.wav 

./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/ESUS/000/A29.wav 903
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/ESUS/000/A30.wav 904
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/ESUS/000/A39.wav 905
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A01.wav 906
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A02.wav 907
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A03.wav 908
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A04.wav 909
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A05.wav 910
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A06.wav 911
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A07.wav 912
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A08.wav 913
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A09.wav 914
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A10.wav 915
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT/000/A12.wav 916
./utterance/pre-stored0.1.2/pre/EJM11/V01/T01/TIMIT

In [7]:
"""
全事前学習出力話者からラムダを推定する.
ラムダは適応学習で変容する.
"""

S = len(total_data)
D = int(total_data[0].shape[1] / 2)
print("total_data[0].shape = ", total_data[0].shape)
print("S = ", S)
print("D = ", D)

timer_start = time.time()
if os.path.exists(pre_stored_gmm_init_pickle):
    print("exist, ", pre_stored_gmm_init_pickle)
    with open(pre_stored_gmm_init_pickle, mode='rb') as f:
        initial_gmm = pickle.load(f)
        print("open, ", pre_stored_gmm_init_pickle)
        print("Load initial_gmm time = ", time.time() - timer_start , "[sec]")  
else:
    initial_gmm = GMM(n_components = Mixtured, covariance_type = 'full')
    initial_gmm.fit(np.vstack(total_data))
    with open(pre_stored_gmm_init_pickle, 'wb') as output:
        pickle.dump(initial_gmm, output)
        print("Make, ", initial_gmm)
        print("Make initial_gmm time = ", time.time() - timer_start , "[sec]")    
            
weights = initial_gmm.weights_
source_means = initial_gmm.means_[:, :D]
target_means = initial_gmm.means_[:, D:]
covarXX = initial_gmm.covars_[:, :D, :D]
covarXY = initial_gmm.covars_[:, :D, D:]
covarYX = initial_gmm.covars_[:, D:, :D]
covarYY = initial_gmm.covars_[:, D:, D:]

fitted_source = source_means
fitted_target = target_means

total_data[0].shape =  (1641, 50)
S =  1012
D =  25
Make,  GMM(covariance_type='full', init_params='wmc', min_covar=0.001,
  n_components=40, n_init=1, n_iter=100, params='wmc', random_state=None,
  tol=0.001, verbose=0)
Make initial_gmm time =  3647.7924370765686 [sec]


In [8]:
"""
SVはGMMスーパーベクトルで、各pre-stored学習における出力話者について平均ベクトルを推定する。
GMMの学習を見てみる必要があるか？
"""

timer_start = time.time()
if os.path.exists(pre_stored_sv_npy):
    print("exist, ", pre_stored_sv_npy)
    sv = np.load(pre_stored_sv_npy)
    print("open, ", pre_stored_sv_npy)
    print("Load pre_stored_sv time = ", time.time() - timer_start , "[sec]")  
    
else:
    sv = []
    for i in range(S):
        gmm = GMM(n_components = Mixtured, params = 'm', init_params = '', covariance_type = 'full')
        gmm.weights_ = initial_gmm.weights_
        gmm.means_ = initial_gmm.means_
        gmm.covars_ = initial_gmm.covars_
        gmm.fit(total_data[i])
        sv.append(gmm.means_)
    sv = np.array(sv)
    np.save(pre_stored_sv_npy, sv)
    print("Make pre_stored_sv time = ", time.time() - timer_start , "[sec]")  

Make pre_stored_sv time =  314.84052205085754 [sec]


In [9]:
"""
各事前学習出力話者のGMM平均ベクトルに対して主成分分析(PCA)を行う.
PCAで求めた固有値と固有ベクトルからeigenvectorsとbiasvectorsを作る.
"""
timer_start = time.time()
#source_pca
source_n_component, source_n_features = sv[:, :, :D].reshape(S, Mixtured*D).shape
# 標準化(分散を1、平均を0にする)
source_stdsc = StandardScaler()
# 共分散行列を求める
source_X_std = source_stdsc.fit_transform(sv[:, :, :D].reshape(S, Mixtured*D)) 

# PCAを行う
source_cov = source_X_std.T @ source_X_std / (source_n_component - 1)
source_W, source_V_pca = np.linalg.eig(source_cov)

print(source_W.shape)
print(source_V_pca.shape)

# データを主成分の空間に変換する
source_X_pca = source_X_std @ source_V_pca
print(source_X_pca.shape)

#target_pca
target_n_component, target_n_features = sv[:, :, D:].reshape(S, Mixtured*D).shape
# 標準化(分散を1、平均を0にする)
target_stdsc = StandardScaler()
#共分散行列を求める
target_X_std = target_stdsc.fit_transform(sv[:, :, D:].reshape(S, Mixtured*D)) 

#PCAを行う
target_cov = target_X_std.T @ target_X_std / (target_n_component - 1)
target_W, target_V_pca = np.linalg.eig(target_cov)

print(target_W.shape)
print(target_V_pca.shape)

# データを主成分の空間に変換する
target_X_pca = target_X_std @ target_V_pca
print(target_X_pca.shape)

eigenvectors = source_X_pca.reshape((Mixtured, D, S)), target_X_pca.reshape((Mixtured, D, S))
source_bias = np.mean(sv[:, :, :D], axis=0)
target_bias = np.mean(sv[:, :, D:], axis=0)
biasvectors = source_bias.reshape((Mixtured, D)), target_bias.reshape((Mixtured, D))

print("Do PCA time = ", time.time() - timer_start , "[sec]")  

(1000,)
(1000, 1000)
(1012, 1000)
(1000,)
(1000, 1000)
(1012, 1000)
Do PCA time =  3.574766159057617 [sec]


In [10]:
"""
声質変換に用いる変換元音声と目標音声を読み込む.
"""

timer_start = time.time()
source_mfcc_for_convert = []
source_sp_for_convert = []
source_f0_for_convert = []
source_ap_for_convert = []
fs_source = None
for name in sorted(glob.iglob(for_convert_source, recursive=True)):
    print("source = ", name)
    x_source, fs_source = sf.read(name)
    f0_source, sp_source, ap_source = analyse_by_world_with_harverst(x_source, fs_source)
    mc = pysptk.sp2mc(sp_source, dim, alpha)
    #mfcc_s_tmp = mfcc_s.mfcc(sp)
    #source_mfcc_for_convert = np.hstack([mfcc_s_tmp, mfcc_s.delta(mfcc_s_tmp)])
    source_mfcc_for_convert.append(mc)
    source_sp_for_convert.append(sp_source)
    source_f0_for_convert.append(f0_source)
    source_ap_for_convert.append(ap_source)

target_mfcc_for_fit = []
target_f0_for_fit = []
target_ap_for_fit = []
for name in sorted(glob.iglob(for_convert_target, recursive=True)):
    print("target = ", name)
    x_target, fs_target = sf.read(name)
    f0_target, sp_target, ap_target = analyse_by_world_with_harverst(x_target, fs_target)
    mc = pysptk.sp2mc(sp_target, dim, alpha)
    #mfcc_target_tmp = mfcc_target.mfcc(sp_target)
    #target_mfcc_for_fit = np.hstack([mfcc_t_tmp, mfcc_t.delta(mfcc_t_tmp)])
    target_mfcc_for_fit.append(mc)
    target_f0_for_fit.append(f0_target)
    target_ap_for_fit.append(ap_target)

# 全部numpy.arrrayにしておく
source_data_mfcc = np.array(source_mfcc_for_convert)
source_data_sp = np.array(source_sp_for_convert)
source_data_f0 = np.array(source_f0_for_convert)
source_data_ap = np.array(source_ap_for_convert)

target_mfcc = np.array(target_mfcc_for_fit)
target_f0 = np.array(target_f0_for_fit)
target_ap = np.array(target_ap_for_fit)

print("Load Input and Target Voice time = ", time.time() - timer_start , "[sec]")  

source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A11.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A14.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A17.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A18.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A19.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A20.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A21.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A22.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A23.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A24.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A25.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A26.wav
source =  ./utterance/pre-stored0.1.2/input/EJM10/V01/T01/TIMIT/000/A27.wav
source =  ./

In [11]:
"""
適応話者学習を行う.
つまり,事前学習出力話者から目標話者の空間を作りだす.

適応話者文数ごとにfitted_targetを集めるのは未実装.
"""

timer_start = time.time()
epoch=100

py = GMM(n_components = Mixtured, covariance_type = 'full')
py.weights_ = weights
py.means_ = target_means
py.covars_ = covarYY

fitted_target = None

for i in range(len(target_mfcc)):
    print("adaptation = ", i+1, "/", len(target_mfcc))
    target = target_mfcc[i]

    for x in range(epoch):
        print("epoch = ", x)
        predict = py.predict_proba(np.atleast_2d(target))
        y = np.sum([predict[:, i: i + 1] * (target - biasvectors[1][i])
                    for i in range(Mixtured)], axis = 1)
        gamma = np.sum(predict, axis = 0)

        left = np.sum([gamma[i] * np.dot(eigenvectors[1][i].T,
                                         np.linalg.solve(py.covars_, eigenvectors[1])[i])
                       for i in range(Mixtured)], axis=0)
        right = np.sum([np.dot(eigenvectors[1][i].T, 
                               np.linalg.solve(py.covars_, y)[i]) 
                        for i in range(Mixtured)], axis = 0)
        weight = np.linalg.solve(left, right)

        fitted_target = np.dot(eigenvectors[1], weight) + biasvectors[1]
        py.means_ = fitted_target
        
print("Load Input and Target Voice time = ", time.time() - timer_start , "[sec]")  

adaptation =  1 / 1
epoch =  0
epoch =  1
epoch =  2
epoch =  3
epoch =  4
epoch =  5
epoch =  6
epoch =  7
epoch =  8
epoch =  9
epoch =  10
epoch =  11
epoch =  12
epoch =  13
epoch =  14
epoch =  15
epoch =  16
epoch =  17
epoch =  18
epoch =  19
epoch =  20
epoch =  21
epoch =  22
epoch =  23
epoch =  24
epoch =  25
epoch =  26
epoch =  27
epoch =  28
epoch =  29
epoch =  30
epoch =  31
epoch =  32
epoch =  33
epoch =  34
epoch =  35
epoch =  36
epoch =  37
epoch =  38
epoch =  39
epoch =  40
epoch =  41
epoch =  42
epoch =  43
epoch =  44
epoch =  45
epoch =  46
epoch =  47
epoch =  48
epoch =  49
epoch =  50
epoch =  51
epoch =  52
epoch =  53
epoch =  54
epoch =  55
epoch =  56
epoch =  57
epoch =  58
epoch =  59
epoch =  60
epoch =  61
epoch =  62
epoch =  63
epoch =  64
epoch =  65
epoch =  66
epoch =  67
epoch =  68
epoch =  69
epoch =  70
epoch =  71
epoch =  72
epoch =  73
epoch =  74
epoch =  75
epoch =  76
epoch =  77
epoch =  78
epoch =  79
epoch =  80
epoch =  81
epoch 

In [12]:
"""
変換に必要なものを残しておく.
"""
np.save(save_for_evgmm_covarXX, covarXX)
np.save(save_for_evgmm_covarYX, covarYX)
np.save(save_for_evgmm_fitted_source, fitted_source)
np.save(save_for_evgmm_fitted_target, fitted_target)
np.save(save_for_evgmm_weights, weights)
np.save(save_for_evgmm_source_means, source_means)